### **계산항 정리** 

$S =$ State의 집합 {$(1,1), (1,2), (1,3)$}

$A =$ Action의 집합

$r^a_s = E[r_{t+1} | S_t = s, A_t = a]$

$P^a_{ss'} = P[S_{t+1} = s' | S_t = s, A_t = a]$

$\gamma$ = Discount Factor

$\pi(a|s) = P[A_t = a | S_t = s]$

$* =$ 최적의 정책  



### **벨만 최적 방정식(해당 state에서 얻을 수 있는 최적의 가치)**
$$v_*(s) = max_aE[r + \gamma{v_*(s')} | s_t = s, a_t = a]$$  
$$v_*(s) = max_aq_*(s, a)$$  
$$q_*(s,a) = r^a_s + \gamma\sum_{s'{\in}S}P^a_{ss'}v_*(s')$$   
$$v_*(s) = max_a\biggl[r^a_s + \gamma\sum_{s'{\in}S}P^a_{ss'}v_*(s')\biggl]$$
$$v_*(s) = max_a\biggl[r^a_s + \gamma\sum_{s'{\in}S}P^a_{ss'}max_{a'}q_*(s',a')\biggl]$$
#### -> 벨만 최적방정식(최적상태함수)은 특정 state에서 특정 action을 했을 때 얻을 수 있는 보상(현재 + 미래) 기댓값 중 가장 큰 원소를 도출하는 것  -> 그냥 최댓값만 고르면 되지 왜 기댓값의 최댓값인가?
##### -> $\pi$에 의한 영향은 사라졌지만 전이확률에 의해 어느 $s'$에 도달할지 알 수 없기 때문에 여전히 기댓값으로 정의해야함


 


**MDP를 알 때**

1. 보상함수  -> state에 따른 penalty와 reward를 알고 있다.(-1, 1 등)

2. 상태전이확률 -> s에서 a를 했을 때 s'으로 전이할 확률을 알고 있다. (1로 고정)

**최고의 정책 찾기**

- 최적 가치를 주는 최적 정책을 찾는다. -> 모든 정책 중에서 가장 많은 보상을 주는 정책이 최적 정책이고 이를 따랐을 때 얻는 가치를 최적 가치라 한다.
- 가치함수 안에 정책이 내재되어 있으므로 가치함수를 업데이트 하면 정책 또한 발전된다.
- 벨만 '최적' 방정식 활용

**policy iteration과의 차이?**
- policy iteration은 최적의 정책을 찾기위해 정책 평가, 개선을 반복한다.
- value iteration은 현재 정책을 최적의 정책으로 가정한다.


In [3]:
import numpy as np
from environment import GraphicDisplay, Env


class ValueIteration:
    def __init__(self, env):
        # 환경에 대한 객체 선언
        self.env = env
        # 가치 함수를 2차원 리스트로 초기화
        self.value_table = [[0.0] * env.width for _ in range(env.height)]
        # 할인율
        self.discount_factor = 0.9

    # 벨만 최적 방정식을 통해 다음 가치 함수 계산
    
    def value_iteration(self):
        # 다음 가치함수 초기화
        next_value_table = [[0.0] * self.env.width 
                           for _ in range(self.env.height)]

        # 모든 상태에 대해서 벨만 최적방정식을 계산
        print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
        print("------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
        for state in self.env.get_all_states():
            # 마침 상태의 가치 함수 = 0 -> 2열 2행이 종료시점이기 때문에 해당 state의 가치함수는 0
            if state == [2, 2]:
                next_value_table[state[0]][state[1]] = 0.0
                continue

            # 벨만 최적 방정식
            value_list = []
            for action in self.env.possible_actions:
                next_state = self.env.state_after_action(state, action)
                reward = self.env.get_reward(state, action)
                next_value = self.get_value(next_state)
                #바로 아랫줄까지는 벨만 기대방정식임(max를 안했으므로)
                value_list.append((reward + self.discount_factor * next_value))
            
            # 최댓값을 다음 가치 함수로 대입
            next_value_table[state[0]][state[1]] = max(value_list)
            print("state = {}의 reward, gamma, next_value: {}, {}, {}".format(state, reward, self.discount_factor, next_value))
            print("state = {}의 value_list (열,행 순서): ".format(state), value_list)
            print("벨만최적방정식을 통해 state = {}의 상태가치함수 최댓값을 새로운 가치함수로 저장: ".format(state), max(value_list))
        self.value_table = next_value_table
        
        

    # 현재 가치 함수로부터 행동을 반환(정책(액션방향)을 출력하는 데 사용)
    def get_action(self, state):
        if state == [2, 2]:
            return []

        # 모든 행동에 대해 큐함수 (보상 + (감가율 * 다음 상태 가치함수))를 계산
        value_list = []
        for action in self.env.possible_actions:
            next_state = self.env.state_after_action(state, action)
            reward = self.env.get_reward(state, action)
            next_value = self.get_value(next_state)
            value = (reward + self.discount_factor * next_value)
            value_list.append(value)

        # 최대 큐 함수를 가진 행동(복수일 경우 여러 개)을 반환
        max_idx_list = np.argwhere(value_list == np.amax(value_list))
        action_list = max_idx_list.flatten().tolist()
        return action_list
        print(action_list)


    def get_value(self, state):
        return self.value_table[state[0]][state[1]]

In [5]:
if __name__ == "__main__":
    env = Env()
    value_iteration = ValueIteration(env)
    grid_world = GraphicDisplay(value_iteration)
    grid_world.mainloop()
# 그리드 월드 칸칸의 값 = 해당 state 이후의 state에서 액션 좌우상하를 했을 때 얻을 수 있는 가치중 최댓값
# e.g. k = 1일때, (3,1)은 v = 0인데, 이는 해당 state에서 발생할 수 있는 value의 최댓값을 의미함 = t+1 reward + (gamma * max(next_value))
# 상태전환확률을 1로 가정하기 때문에, 좌로 이동시 -> -1 + (0.9 * 0) = -1
# 상태전환확률을 1로 가정하기 때문에, 우로 이동시 -> 0 + (0.9 * 0) = 0
# 상태전환확률을 1로 가정하기 때문에, 상으로 이동시 -> 0 + (0.9 * 0) = 0
# 상태전환확률을 1로 가정하기 때문에, 하로 이동시 -> 0 + (0.9 * 0) = 0
# [-1.0, 0.0, 0.0, 0.0]
# 해당 state의 k = 1 기준 max(상하좌우v) = 0

# e.g. k = 2일때, (3,1)은 v = 0.9인데, 
# 상태전환확률을 1로 가정하기 때문에, 좌로 이동시 -> -1 + (0.9 * 1) = -0.1
# 상태전환확률을 1로 가정하기 때문에, 우로 이동시 -> 0 + (0.9 * 0) = 0
# 상태전환확률을 1로 가정하기 때문에, 상으로 이동시 -> 0 + (0.9 * 0) = 0
# 상태전환확률을 1로 가정하기 때문에, 하로 이동시 -> 0 + (0.9 * 1) = 0.9
#[-0.1, 0.0, 0.0, 0.9]
# 해당 state의 k = 2 기준 max(상하좌우v) = 0.9

# k = 2에서의 state(3,1)의 v로 인해 k = 3 시점에서는 state(3,0)의 v가 0.81로 산출됨(이전에는 계속 0)
# 상태전환확률을 1로 가정하기 때문에, 좌로 이동시 -> 0 + (0.9 * 0) = 0
# 상태전환확률을 1로 가정하기 때문에, 우로 이동시 -> 0 + (0.9 * 0) = 0
# 상태전환확률을 1로 가정하기 때문에, 상으로 이동시 -> 0 + (0.9 * 0) = 0
# 상태전환확률을 1로 가정하기 때문에, 하로 이동시 -> 0 + (0.9 * 0.9) = 0.81
#[0.0, 0.0, 0.0, 0.81]

#이 과정을 무한 반복하면 결국 모든 지점에서의 가치함수를 산출할 수 있다.


# 상태전환확률을 1로 가정하기 때문에, 1) 0.25 * (-1 + (0.9 * 0)) = 0.25
# 모두 훈련하고 난 후 state 0, 0 기준으로 벨만기대방정식 계산값은 0.59인데
# 이는 0,0기준 오른쪽으로 이동했을 때의 기댓값 0.5 * (0 + (0.9 * 0.66)) + 아래로 이동했을 때의 기댓값 0.5 * (0 + (0.9 * 0.66)) 을 더해서 0.594가 나오는 것

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
state = [0, 0]의 reward, gamma, next_value: 0, 0.9, 0.0
state = [0, 0]의 value_list (열,행 순서):  [0.0, 0.0, 0.0, 0.0]
벨만최적방정식을 통해 state = [0, 0]의 상태가치함수 최댓값을 새로운 가치함수로 저장:  0.0
state = [0, 1]의 reward, gamma, next_value: 0, 0.9, 0.0
state = [0, 1]의 value_list (열,행 순서):  [0.0, 0.0, 0.0, 0.0]
벨만최적방정식을 통해 state = [0, 1]의 상태가치함수 최댓값을 새로운 가치함수로 저장:  0.0
state = [0, 2]의 reward, gamma, next_value: 0, 0.9, 0.0
state = [0, 2]의 value_list (열,행 순서):  [0.0, -1.0, 0.0, 0.0]
벨만최적방정식을 통해 state = [0, 2]의 상태가치함수 최댓값을 새로운 가치함수로 저장:  0.0
state = [0, 3]의 reward, gamma, next_value: 0, 0.9, 0.0
state = [0, 3]의 value_list (열,행 순서):  [0.0, 0.0, 0.